A method that is called implicitly by Python to execute a certain operation on a type, such as addition. Such methods have names starting and ending with double underscores.

In [1]:
# When we do 1+2, it works like:
(1).__add__(2)

3

#### Controlling Object creation process

In [2]:
# init - used to initialize the object

class Demo:
    def __init__(self,name):
        self.name = name
    
    def disp(self):
        print(f"Name: {self.name}")

demo = Demo("satya")
demo.disp()

Name: satya


In [4]:
# new - used to initialize the class which is then passed to init

class Demo:
    def __new__(cls):
        print("Creating Demo instance")
        return super().__new__(cls)
    
    def hello(self):
        print("Hello")
    
demo = Demo()
demo.hello()

Creating Demo instance
Hello


#### Objects as strings

In [5]:
class Demo:
    def __init__(self,name):
        self.name = name
    
    def disp(self):
        print(f"Name: {self.name}")

demo = Demo("satya")
print(demo)
demo

In [7]:
# the above are printed and returned when object is printed or called. to change this is 

class Demo:
    def __init__(self,name):
        self.name = name
    
    def disp(self):
        print(f"Name: {self.name}")

    # Informal Representation
    def __str__(self) -> str:
        return "Demo printed"
    
    # Formal Representation
    def __repr__(self) -> str:
        return "Demo called"

demo = Demo("satya")
print(demo)
demo

Demo printed


Demo called

#### Operator Overloading

In [47]:
class Storage():
    def __init__(self,value,unit) -> None:
        self.value = value
        self.unit = unit
    
    def __add__(self,other):
        if(not isinstance(other,type(self))):
            raise TypeError("Unsupported +")
        
        if self.unit != other.unit:
            raise TypeError("Unsupported units")
        
        return Storage(self.value + other.value, self.unit)

In [48]:
disk1 = Storage(1000,"GB")
disk2 = Storage(2000,"GB")
disk3 = Storage(3,"TB")

res1 = disk1 + disk2
res1.value, res1.unit

(3000, 'GB')

In [49]:
res2 = disk1 + disk3

TypeError: Unsupported units

When we use add, it does the operation taking the first basis object type from left, that is, if we have obj + obj and if the left obj is not of any numeric type then this would fail  
To overcome this, python had \_\_r*\_\_ version of these arithmetic operations

In [67]:
class Number:
    def __init__(self, value) -> None:
        self.value = value
    
    def __add__(self, other):
        print("add called")
        if(isinstance(other,Number)):
            return Number(self.value + other.value)
        elif(isinstance(other,int or float)):
            return Number(self.value + other)
        else:
            raise TypeError("Unsupported types for +")

If we normally do 1+2, it's fine bcs both are numeric, but if we do some `number + instance_obj`,then add is called bcs first operand is numeric and add method is defined is numeric type.  
But if `instance_obj + number` is called, it gets rejected because first operand doesnt(maybe) has add defined.  

In [68]:
one = Number(1)
two = Number(2)

three = one + two 
print(three.value)

add called
3


In [69]:
five = three + 2
five.value

add called


5

In [66]:
six = 1 + five

TypeError: unsupported operand type(s) for +: 'int' and 'Number'

The above doesn't work as we have no implementation to check if add works for int + Number in the first operand, i.e 1  
So we add another method of radd which supports right first 

In [70]:
class Number:
    def __init__(self, value) -> None:
        self.value = value
    
    def __add__(self, other):
        print("add called")
        if(isinstance(other,Number)):
            return Number(self.value + other.value)
        elif(isinstance(other,int or float)):
            return Number(self.value + other)
        else:
            raise TypeError("Unsupported types for +")
    
    def __radd__(self,other):
        print("radd called")
        return self.__add__(other)

In [72]:
five = Number(5)
six = 1 + five
six.value

radd called
add called


6

In [73]:
one = Number(1)
seven = one + six
seven.value

add called


7

In [74]:
seven = six + 1
seven.value

add called


7

In [75]:
seven = 1 + six
seven.value

radd called
add called


7

The reason your first code doesn't work when adding an `int` to a `Number` object (i.e., `1 + five`) but works for `five + 1` is due to Python's internal mechanism for handling binary operations like addition (`+`).

Here's what happens in each case:

##### Case 1: `five + 1`
In this case, Python tries to call the `__add__` method on the `Number` object (`five`), which works because you've defined `__add__` in the `Number` class. The method checks if the other operand (`1`) is an `int` or `float` and returns the correct result.

##### Case 2: `1 + five`
This is where it doesn't work without the `__radd__` method. Here’s why:
1. Python first tries to call `__add__` on the `int` object (`1`). Since `int` does not know how to add itself to a `Number` object, it fails.
2. When Python can't perform the addition from the left-hand side (`1`), it looks for a method called `__radd__` (right-side addition) in the `Number` class.
3. Since the `Number` class didn’t have an `__radd__` method in your first implementation, Python throws an error. But after you added the `__radd__` method, Python knows how to handle the operation.

##### Solution: The `__radd__` Method

The second code works because you define the `__radd__` method. Python calls this method when the left operand doesn't know how to handle the operation (like when trying to add an `int` and a `Number`).

##### Key Points:
- **`__add__`**: Handles the `Number + int` or `Number + Number` case.
- **`__radd__`**: Handles the `int + Number` or `float + Number` case, which is needed when the first operand doesn’t support adding a `Number`.

In Python, this mechanism ensures that you can perform operations regardless of the order of the operands (`Number + int` or `int + Number`).